# Helper Functions

## Table of Contents

1. Create S3 bucket
2. Upload file to S3 bucket
3. Download file from S3 bucket


In [2]:
# create a s3 bucket
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/create_bucket.html#create-bucket
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")


def create_s3_bucket(bucket_name, region="us-east-1"):
    """
    Creates an S3 bucket in the specified region.

    Parameters:
    - bucket_name (str): The name of the bucket to be created.
    - region (str, optional): The AWS region in which to create the bucket. Default is 'us-east-1'.

    Returns:
    - A tuple of (bool, dict). The first element is a boolean indicating whether the bucket was created and the second element
        is the response from the S3 API if the bucket was created successfully, otherwise None.
    """
    s3 = boto3.client(
        "s3",
        region_name=region,
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
    )

    # Check if the bucket name is available
    try:
        s3.head_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' already exists.")
        return False, None
    except Exception as e:
        if e.response["Error"]["Code"] == "404":
            # Create the bucket
            if region == "us-east-1":
                response = s3.create_bucket(Bucket=bucket_name)
            else:
                response = s3.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={"LocationConstraint": region},
                )
            print(f"Bucket '{bucket_name}' created successfully.")
            return True, response
        else:
            # Other errors occurred
            print(f"Error: {e}")
            return False, None

In [3]:
# Example usage
bucket_name = "pmd-lambda-outputs"
region = "us-east-1"
create_s3_bucket(bucket_name, region)

Bucket 'pmd-lambda-outputs' already exists.


(False, None)

In [5]:
# Example usage
bucket_name = "pmd-lambda-layers"
region = "us-east-1"
create_s3_bucket(bucket_name, region)

Bucket 'pmd-lambda-layers' created successfully.


(True,
 {'ResponseMetadata': {'RequestId': '7GWZTDQXSJGA00E8',
   'HostId': '1QIPcd0U+TkTnbStn3JQxmQy2cHN0CYW6aN6X022r9jjXYQFZEvpXjzlHuzCYpaTWlY4U8xdZa4=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '1QIPcd0U+TkTnbStn3JQxmQy2cHN0CYW6aN6X022r9jjXYQFZEvpXjzlHuzCYpaTWlY4U8xdZa4=',
    'x-amz-request-id': '7GWZTDQXSJGA00E8',
    'date': 'Mon, 03 Jul 2023 19:11:12 GMT',
    'location': '/pmd-lambda-layers',
    'server': 'AmazonS3',
    'content-length': '0'},
   'RetryAttempts': 0},
  'Location': '/pmd-lambda-layers'})

In [6]:
def upload_file_to_s3(
    local_file_path, bucket_name, s3_file_key=None, buffered_data=None
):
    """
    Uploads a file to Amazon S3.

    Parameters:
    - local_file_path (str): The local file path of the file to upload.
    - bucket_name (str): The name of the S3 bucket to upload the file to.
    - s3_file_key (str, optional): The S3 file key (path) where the file will be stored. If not provided,
                                  the file will be stored in the root of the bucket with the same name as the
                                  local file.
    - buffered_data (bytes, optional): The buffered data to upload instead of using a local file. If provided,
                                       local_file_path will be ignored.

    Returns:
    - The S3 URL of the uploaded file.
    """
    s3 = boto3.client(
        "s3",
        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
        region_name=os.environ.get("AWS_REGION"),
    )
    if buffered_data:
        response = s3.put_object(
            Bucket=bucket_name, Key=s3_file_key, Body=buffered_data
        )
    else:
        # If s3_file_key is not provided, use the local file name as the key
        if s3_file_key is None:
            s3_file_key = local_file_path.split("/")[-1]

        # Upload the file to S3
        with open(local_file_path, "rb") as file:
            response = s3.put_object(Bucket=bucket_name, Key=s3_file_key, Body=file)

    # Get the URL of the uploaded file
    s3_url = f"https://{bucket_name}.s3.amazonaws.com/{s3_file_key}"
    return s3_url


In [8]:
upload_file_to_s3(
    "..\\lambdas\\lambda2\\layers\\texttospeechlayer.zip",
    "pmd-lambda-layers",
    "texttospeechlayer.zip",
)

'https://pmd-lambda-layers.s3.amazonaws.com/texttospeechlayer.zip'

In [38]:
import boto3
from pathlib import Path


def download_file_from_s3(s3_url, local_file_path):
    """
    Downloads a file from an S3 URL to a local file.

    Parameters:
    - s3_url (str): The URL of the file in S3, e.g., 'https://your-bucket-name.s3.amazonaws.com/your-file.txt'.
    - local_file_path (str): The local file path where the downloaded file will be stored.

    Returns:
    - True if the download is successful, False otherwise.
    """
    s3 = boto3.client(
        "s3",
        region_name=os.environ.get("AWS_REGION"),
        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    )

    # Extract the bucket name and object key from the S3 URL
    try:
        bucket_name = s3_url.split("//")[1].split(".")[0]
        object_key = s3_url.split(bucket_name + ".s3.amazonaws.com/")[1]
    except Exception as e:
        print("Error parsing the S3 URL:", e)
        return False
    print(f"Bucket name: {bucket_name}, Object key: {object_key}")
    if local_file_path == "":
        local_file_path = object_key
    # create directory if not exists
    Path(local_file_path).parent.mkdir(parents=True, exist_ok=True)
    # Download the file from S3
    try:
        s3.download_file(bucket_name, object_key, local_file_path)
        print(f"File downloaded successfully to {local_file_path}.")
        return True
    except Exception as e:
        print("Error downloading the file:", e)
        return False


# Example usage
s3_url = "https://pmd-lambda-outputs.s3.amazonaws.com/audio_files/sample_audio.wav"
local_file_path = ""
download_file_from_s3(s3_url, local_file_path)


Bucket name: pmd-lambda-outputs, Object key: audio_files/sample_audio.wav
File downloaded successfully to audio_files/sample_audio.wav.


True